<a href="https://colab.research.google.com/github/knobs-dials/wetsuite-dev/blob/main/notebooks/intro/using_dataset_kamervragen.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# (only) in colab, run this first to install wetsuite from (the most recent) source.   For your own setup, see wetsuite's install guidelines.
!pip3 install -U --no-cache-dir --quiet https://github.com/scarfboy/wetsuite-dev/archive/refs/heads/main.zip


In [1]:
import json
import wetsuite.datasets
tkf = wetsuite.datasets.load('tweedekamer_fracties')
tkm = wetsuite.datasets.load('tweedekamer_membership')

#with open('/var/www/coding/wetsuite/src/wetsuite/datacollect/tweedekamer_nl/fracties.json','r') as f:
#    fracties = json.loads( f.read() )
    
#with open('/var/www/coding/wetsuite/src/wetsuite/datacollect/tweedekamer_nl/fractie_membership.json','r') as f:
#    fractie_membership = json.loads( f.read() )
        

In [41]:
from wetsuite.helpers.strings import remove_diacritics


def mostly_like(s1:str, s2:str):
    return s1 == s2


def remove_surnameprefix(s:str, what=['van de', 'van der', 'van den', "van 't", 'van']):
    import re
    for w in what:
        #print( s, w )
        if re.match( w + r'\b',  s ):
            s = s[len(w):].strip()
            break
    return s


def fuzzy_name_search(first:str, last:str=None):
    if last is None: # got just one string, see if we can treat it as 'first last'
        #first = remove_surnameprefix( first )
        fl  = first.split(' ', 1)
        #print(fl)
        if len(fl)==1: # assume it's last name only
            first=None
            last = fl[0]
        else: # len is 2
            first, last = fl

    #print('f1',first, 'l1',last)

    if first is not None:
        first = remove_diacritics( first )
    last  = remove_diacritics( last )
    last = remove_surnameprefix( last )

    print('Looking for f2:%r l2:%r'%(first, last))

    first_is_missing = (first is None)
    first_is_initial = False
    if not first_is_missing: 
        if len(first.strip('. ')) == 1:
            first_is_initial = True

    for d in fractie_membership.values():
        #print( d )
        if first_is_missing:
            pass
        elif first_is_initial:
            if not mostly_like( first,  remove_diacritics( d['initialen'].strip('. ') ) ):
                continue
        else:
            dfirst = d['voornamen']
            if dfirst==None:
                dfirst = d['roepnaam']
            if not mostly_like( first,  remove_diacritics( dfirst ) ):
                continue
        print('passed first: ', d)

        if not mostly_like( last,  remove_surnameprefix(remove_diacritics( d['achternaam'] ) ) ):
            continue

        print('Search %r %r  seems to match  %r'%(first, last, d))




#fuzzy_name_search('Haersma')
#fuzzy_name_search('Sybrand Haersma')
#fuzzy_name_search('Sybrand', 'Haersma')
#fuzzy_name_search('Sybrand', 'van Haersma Buma')
fuzzy_name_search('van Haersma Buma')
#fuzzy_name_search('S', 'Haersma')
#fuzzy_name_search('S', 'van Haersma')
#fuzzy_name_search('S', 'van Haersma Buma')
#fuzzy_name_search('Sybren', 'van Haersma Buma')


# 'titels': 'mr.',
# 'initialen': 'S.',
# 'voornamen': 'Sybrand',
# 'roepnaam': 'Sybrand',

# 'tussenvoegsel': 'van',
# 'achternaam': 'Haersma Buma',



Looking for f2:'Haersma' l2:'Buma'
